## 0. Load imports 

In [193]:
## imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 0. Load data

In [194]:
## load data on 2020 crimes in DC
df = dc_crim_2020 = pd.read_csv("https://opendata.arcgis.com/datasets/f516e0dd7b614b088ad781b0c4002331_2.csv")
# dc_crim_2020.head()
# dc_crim_2020.shape
# dc_crim_2020.info()


In [168]:
df.OFFENSE.value_counts()

OFFENSE
THEFT/OTHER                   10927
THEFT F/AUTO                   8283
MOTOR VEHICLE THEFT            3260
ROBBERY                        1998
ASSAULT W/DANGEROUS WEAPON     1632
BURGLARY                       1444
HOMICIDE                        198
SEX ABUSE                       178
ARSON                            13
Name: count, dtype: int64

In [ ]:
df.OFFENSE.value_counts()

In [14]:
## Use tab to autocomplete
## Use shift-tab to see variables in a function.

df.OFFENSE.value_counts()
df.WARD.value_counts()

In [99]:
df.X + 1

0       -76.004500
1       -75.972651
2       -75.972226
3       -76.023981
4       -76.044182
           ...    
27928   -76.024529
27929   -75.982318
27930   -76.053302
27931   -76.028143
27932   -75.961475
Name: X, Length: 27933, dtype: float64

#### Warm-up: Ways to add one to the column

In [197]:
df.X + 1

0       -76.004500
1       -75.972651
2       -75.972226
3       -76.023981
4       -76.044182
           ...    
27928   -76.024529
27929   -75.982318
27930   -76.053302
27931   -76.028143
27932   -75.961475
Name: X, Length: 27933, dtype: float64

In [ ]:
plus_1 = lambda x: x + 1

def plus_1_v2(x):
    return x + 1

In [198]:
df.X.apply(plus_1)

0       -76.004500
1       -75.972651
2       -75.972226
3       -76.023981
4       -76.044182
           ...    
27928   -76.024529
27929   -75.982318
27930   -76.053302
27931   -76.028143
27932   -75.961475
Name: X, Length: 27933, dtype: float64

In [106]:
## To see what a sub-dataframe looks like, iterate with a for-loop

for i, ddf in df.groupby("WARD"):
    break
ddf[["WARD", "OFFENSE", "SHIFT"]]

,WARD,OFFENSE,SHIFT
12,1,THEFT/OTHER,DAY
23,1,THEFT F/AUTO,MIDNIGHT
27,1,THEFT/OTHER,EVENING
28,1,THEFT F/AUTO,DAY
30,1,THEFT/OTHER,EVENING
...,...,...,...
27899,1,THEFT F/AUTO,DAY
27915,1,MOTOR VEHICLE THEFT,MIDNIGHT
27916,1,THEFT F/AUTO,EVENING
27920,1,THEFT F/AUTO,DAY


## 1. Aggregation

### Example of grouping by one variable and doing one aggregation 

**Task**: find and print the number of unique offense types (`OFFENSE`) by ward (`WARD`)

In [177]:
## Explore data by simulating a loop / groupby
for i, ddf in df.groupby("WARD"):
    break
    
y = ddf[["WARD", "OFFENSE"]].OFFENSE.unique()
len(y)

9

In [118]:
## Translate exploration code into a function
def get_unique_num(X):
    y = X.unique()
    return len(y)

In [203]:
df.groupby(["WARD"]).agg( 
    {"OFFENSE": get_unique_num}
).reset_index() #the iloc shows the first five

,WARD,OFFENSE
0,1,9
1,2,9
2,3,8
3,4,8
4,5,9
5,6,9
6,7,9
7,8,9


In [179]:
%%time
uniq_off_byward = U = df.groupby("WARD").agg(
    {"OFFENSE":"nunique"}
    ).reset_index()
U

CPU times: user 16.1 ms, sys: 2.4 ms, total: 18.5 ms
Wall time: 21 ms


,WARD,OFFENSE
0,1,9
1,2,9
2,3,8
3,4,8
4,5,9
5,6,9
6,7,9
7,8,9


### Example of grouping by one variable and providing two summaries of the same variable

**Task**: previous showed number of offenses by ward but want to find out content of offenses in each ward; create an aggregation that summarizes both the number of unique offenses by ward and what those offenses are -- for instance, by pasting the unique offenses in that ward separated by the ";" (e.g., Theft; Burglary;...)

*Hint*: you can use the join command to paste together a list separated by some delimiter. 
    - The syntax, if we are using the comma delimiter, is: ", ".join(nameoflist)

In [46]:
def get_uniq_offs(X):
    uniq_offs = sorted( X.unique() )
    uniq_offs = "; ".join( uniq_offs )
    return uniq_offs

In [183]:
A = uniq_off_valcount_byward = df.groupby("WARD").agg(
        {"OFFENSE": ["nunique", get_uniq_offs]}
        )
A

OFFENSE                                                   
     nunique                                      get_uniq_offs
WARD                                                           
1          9  ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...
2          9  ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...
3          8  ASSAULT W/DANGEROUS WEAPON; BURGLARY; HOMICIDE...
4          8  ASSAULT W/DANGEROUS WEAPON; BURGLARY; HOMICIDE...
5          9  ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...
6          9  ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...
7          9  ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...
8          9  ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...

### Example of grouping by two variables 

**Task**: group by ward (`WARD`) and police shift (`SHIFT`) and find the offense that is most common in that ward and shift

In [187]:
# Explore the data
for i, ddf in df.groupby(["WARD", "SHIFT"]):
    break
    
x = ddf.OFFENSE
x = x.value_counts()
x.index[0]

'THEFT/OTHER'

In [189]:
df.groupby(["WARD", "SHIFT"])["OFFENSE"].agg(pd.Series.mode).reset_index()

,WARD,SHIFT,OFFENSE
0,1,DAY,THEFT/OTHER
1,1,EVENING,THEFT/OTHER
2,1,MIDNIGHT,THEFT/OTHER
3,2,DAY,THEFT/OTHER
4,2,EVENING,THEFT/OTHER
5,2,MIDNIGHT,THEFT/OTHER
6,3,DAY,THEFT/OTHER
7,3,EVENING,THEFT/OTHER
8,3,MIDNIGHT,THEFT/OTHER
9,4,DAY,THEFT F/AUTO


In [188]:
B = ward_shift_top_offense = df.groupby(["WARD", "SHIFT"]).agg(
        {
            "OFFENSE": lambda x: x.value_counts().index[0]
        }
        ).reset_index()
B

,WARD,SHIFT,OFFENSE
0,1,DAY,THEFT/OTHER
1,1,EVENING,THEFT/OTHER
2,1,MIDNIGHT,THEFT/OTHER
3,2,DAY,THEFT/OTHER
4,2,EVENING,THEFT/OTHER
5,2,MIDNIGHT,THEFT/OTHER
6,3,DAY,THEFT/OTHER
7,3,EVENING,THEFT/OTHER
8,3,MIDNIGHT,THEFT/OTHER
9,4,DAY,THEFT F/AUTO


## Summarizing over all rows or all columns (without grouping)

We can also use the `apply` function to summarize rows or columns efficiently

**Task**: find the mean lat and longitude in one line of code

In [63]:
df[["LATITUDE", "LONGITUDE"]].apply("mean", axis = 0)

LATITUDE     38.907613
LONGITUDE   -77.007049
dtype: float64

## 2. Creating new columns/transforming their type 

### Simple filtering

`np.where` is in the numpy package (aliased as `np`) and operates similar to `ifelse` in R

There is a built in pandas way to emulate this.

**Task**: create an indicator `is_theft` for any offense that contains the word "THEFT"
    
**Task**: create an indicator `is_theft_notmotor` for any offense that contains the word "THEFT" but does not contain the word "MOTOR"

In [158]:
df["is_theft"] = df.OFFENSE.str.contains("THEFT")

In [69]:
df["is_theft_notmotor"] = ( df.OFFENSE.str.contains("THEFT") &
                        ~df.OFFENSE.str.contains("MOTOR")  )

In [159]:
df.is_theft & (~df.OFFENSE.str.contains("MOTOR"))

0        False
1        False
2        False
3         True
4        False
         ...  
27928    False
27929    False
27930     True
27931    False
27932     True
Length: 27933, dtype: bool